# Caclulate statistics per district using DuckDB

**Author**: Willeke A'Campo

**Description:** This notebooks shows how to calculate the Ecosystem Service statistics per district using DuckDB. The results are stored in a new table in the database and exported to GeoJSON.

**Documentation:** 

### Data conversion | GeoJSON to GeoParquet   

In [1]:
import geopandas as gpd
from shapely.geometry import Point
import pyarrow
import os
import leafmap
import os
import duckdb
import pandas as pd

municipality = "kristiansand"
raw_dir = r"/workspaces/urban-climate/data/01_raw"
interim_dir = r"/workspaces/urban-climate/data/02_intermediate"

# Define the table names
file_names = [
    f"{municipality}_study_area", 
    #f"{municipality}_districts",
    f"{municipality}_bldg",
    f"{municipality}_res_bldg",
    f"{municipality}_open_space",
    f"{municipality}_public_open_space",
    f"{municipality}_private_open_space",
    f"{municipality}_tree_crowns"
    ]

table_names = [
    "study_area", "buildings", "residential_buildings", 
    "open_space", "public_open_space", "private_open_space", "crowns"
    ]

# Define the parquet_dict
parquet_dict = {
    name: os.path.join(interim_dir, f"{name}.parquet") 
    for name in table_names}

# Check if the parquet files exist, if not convert  to parquet
for key in parquet_dict.keys():
    if os.path.exists(parquet_dict[key]):
        continue
    else:
        # Define the gdf_dict
        gdf_dict = {
            name: gpd.read_file(os.path.join(raw_dir, f"{name}.geojson")) 
            for name in file_names}

        # Convert GeoDataFrame to Parquet
        for key, gdf in gdf_dict.items():
            gdf.to_parquet(
                path = interim_dir + "/" + key + ".parquet",
                index = None, 
                compression = "snappy"
            )

In [2]:
# Create a connection to the DuckDB database
con = duckdb.connect(database=":memory:", read_only=False)
con.install_extension("spatial")
con.load_extension("spatial")

# Create a table for each parquet file or GeoDataFrame
for key in parquet_dict.keys():
    con.execute(
        f"""
        CREATE TABLE {key} 
        AS SELECT *, ST_GeomFromWKB(geometry) 
        FROM parquet_scan('{parquet_dict[key]}')
        """
        )
    

# Fetch and print all table names
result = con.execute(
    """
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'main'
    """
    )

print(result.fetchall())

[('study_area',), ('buildings',), ('residential_buildings',), ('open_space',), ('public_open_space',), ('private_open_space',), ('crowns',)]


In [21]:
# Check if the 'crowns' table exists
table_exists = "crowns" in [
    row[0]
    for row in con.execute(
        """
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'main'
        """
    ).fetchall()
]

if table_exists:
    # convert dtype to DuckDB GEOMETRY
    result = con.execute(
        """
        SELECT 
        ST_X(ST_Centroid(ST_GeomFromWKB(geometry))), 
        ST_Y(ST_Centroid(ST_GeomFromWKB(geometry))) 
        FROM crowns"""
        )
    # xy_crowns to pd
    df = pd.DataFrame(result.fetchall(), columns=["X", "Y"])
    
    xy_crowns = gpd.GeoDataFrame(
        df,
        geometry= gpd.points_from_xy(df.X, df.Y)
        )
    xy_crowns.crs = "EPSG:25832"

In [38]:
# add layers to gdf for mapping
gdf_study_area = leafmap.read_parquet(
    parquet_dict["study_area"], 
    return_type='gdf', 
    src_crs="EPSG:25832", 
    dst_crs="EPSG:4326"
    )

# convert xy_crowns to wgs84
xy_crowns_sample = xy_crowns.sample(frac=0.05)
trees_xy = xy_crowns_sample.to_crs("EPSG:4326")
points_geojson = trees_xy.__geo_interface__
print(trees_xy.head())

# Calculate the center of the study_area GeoDataFrame
center = gdf_study_area.geometry.unary_union.centroid

# --------------------------------------------------
# INIT MAP
# --------------------------------------------------
map = leafmap.Map()
# center
map.set_center(center.x, center.y, zoom=14)
# add Basemap
map.add_basemap("CartoDB.Positron")

# add study area as vector layer
map.add_gdf(
        gdf_study_area, 
        layer_name="study_area", 
        get_fill_color=[0, 0, 255, 128]
        )

map.add_gdf(
    trees_xy,
    layer_name ="trees",
    color= "black"
)

map.add_legend(
                legend_title="Legend", 
                legend_dict={
                    "Study area": "blue", 
                    "Tree crowns(10% sample)": "blue"
                    }
                )

map

                  X             Y                  geometry
8863  442059.521199  6.445464e+06  POINT (8.01572 58.14669)
8435  441996.894925  6.445853e+06  POINT (8.01456 58.15017)
968   439238.406645  6.446046e+06  POINT (7.96766 58.15153)
342   440180.823424  6.445527e+06  POINT (7.98380 58.14700)
846   439494.722936  6.446085e+06  POINT (7.97200 58.15192)


Map(center=[58.15207603451838, 8.004060328655429], controls=(ZoomControl(options=['position', 'zoom_in_text', …

### DuckDB Connection

You can use the following commands to load your data into DuckDB database
- `con.sql` executes a SQL query writes it to a DuckDB database 
- `con.execute` executes a SQL query and returns the result as a duckdb.DuckDBPyResult object
   - con.execut("CREATE TABLE ...") to create a new table
   - con.execute("INSERT INTO ...") to insert data into a table
   - con.execute("COPY ... FROM ...") to load data from a file into a table
- `gdf.to_sql` to write a GeoDataFrame to a table in the DuckDB database

con.execute("""
COPY table_name 
FROM 'path_to_your_file.parquet' 
WITH (FORMAT 'PARQUET')
""")

**Create a new Table with Tree Crown Center Points**

**Split open space, private open space and public open space by district**

In [37]:
# create new table split_open_space 
# with open space split by district boundaries
con.execute(
    """
    CREATE TABLE split_open_space AS 
    SELECT
        districts.grunnkretsnummer,
        ST_Intersection(districts.geometry, open_space.geometry) AS geom
    FROM 
        districts, open_space
    WHERE
        ST_Intersects(districts.geom, open_space.geom)
    """
    )

# create new table split_buildings
# with buildings split by district boundaries
con.execute(
    """
    CREATE TABLE split_buildings AS 
    SELECT
        districts.grunnkretsnummer,
        ST_Intersection(districts.geometry, buildings.geometry) AS geom
    FROM 
        districts, buildings
    WHERE
        ST_Intersects(districts.geom, buildings.geom)
    """
    )

# create new table split_res_buildings
# with residential buildings split by district boundaries
con.execute(
    """
    CREATE TABLE split_res_buildings AS 
    SELECT
        districts.grunnkretsnummer,
        ST_Intersection(districts.geometry, residential_buildings.geometry) AS geom
    FROM 
        districts, residential_buildings
    WHERE
        ST_Intersects(districts.geom, residential_buildings.geom)
    """
    )

CatalogException: Catalog Error: Table with name districts does not exist!
Did you mean "crowns"?

### Generate Columns with Count Statistics

| Name | Alias | Description | Type |  Unit | 
| --- | --- | --- | --- | --- |
| n_trees | Antall trær | Number of trees in the district | INT |
| n_bldg | Antall bygninger | Number of buildings in the district | INT |
| n_res_bldg | Antall boliger | Number of residential buildings in the district | INT |
| n_res_bldg_near_gs | Antall boliger nær grøntområde (300 m) | Number of residential buildings near green space (300 m) | INT |
| n_trees_near_bldg | Antall trær nær boliger (15 m) | Number of trees near residential buildings (15 m) | INT |
| n_viewshed | Antall viewshed piksler | Number of viewshed pixels that intersect with the building edge | INT |



In [ ]:
# add count columsn to district table 

# add count columns: n_trees, n_bldg, n_res_bldg, n_res_bldg_near_gs, n_trees_near_bldg
con.execute(
    """
    ALTER TABLE districts
    ADD COLUMN n_trees INTEGER,
    ADD COLUMN n_bldg INTEGER,
    ADD COLUMN n_res_bldg INTEGER,
    ADD COLUMN n_res_bldg_near_gs INTEGER,
    ADD COLUMN n_trees_near_bldg INTEGER
    """
    )

# calculate count per district using geometry intersection
con.execute(
    """
    UPDATE districts SET
    n_trees = (
        SELECT COUNT(*) 
        FROM crowns_xy 
        WHERE ST_Within(geom, districts.geom)
        ),
    n_bldg = (
        SELECT COUNT(*) 
        FROM split_buildings 
        WHERE ST_Within(geom, districts.geom)
        ),
    n_res_bldg = (
        SELECT COUNT(*) 
        FROM split_res_buildings 
        WHERE ST_Within(geom, districts.geom)
        ),
    n_res_bldg_near_gs = (
        SELECT COUNT(*) 
        FROM split_res_buildings 
        WHERE ST_DWithin(geom, (
            SELECT geom 
            FROM crowns_xy 
            WHERE ST_Within(geom, districts.geom)), 300)
            ),
    n_trees_near_bldg = (
        SELECT COUNT(*) 
        FROM crowns_xy 
        WHERE ST_DWithin(geom, (
            SELECT geom 
            FROM split_buildings 
            WHERE ST_Within(geom, districts.geom)), 15)
            )
    """
    )

### Generate Columns with Area Statistics

| Name | Alias | Description | Type |  Unit |
| --- | --- | --- | --- | --- |
| a_district | Grunnkretsareal | Area of the district | FLOAT | m2 |
| a_open_space | Åpent område | Area of open space | FLOAT | m2 |
| a_private_space | Privat område | Area of private space | FLOAT | m2 |
| a_public_space | Offentlig område | Area of public space | FLOAT | m2 |
| a_green_space | Grøntområde | Area of green space | FLOAT | m2 |
| a_crown | Kroneareal | Crown coverage area within the district | FLOAT | m2 |
| a_crown_public | Kroneareal i offentlig område | Crown coverage area within public space | FLOAT | m2 |
| a_crown_private | Kroneareal i privat område | Crown coverage area within private space | FLOAT | m2 |


In [ ]:


# Assuming you have already loaded your tables into DuckDB
# You can use con.execute("CREATE TABLE ...") and con.execute("COPY ... FROM ...") to load your data

# Update district_polygons table
con.execute("""
    UPDATE district_polygons SET
        n_trees = (SELECT COUNT(*) FROM tree_crown_polygons WHERE ST_Within(geom, district_polygons.geom)),
        n_building = (SELECT COUNT(*) FROM building_polygons WHERE ST_Within(geom, district_polygons.geom)),
        n_res_bldg = (SELECT COUNT(*) FROM residential_building_polygons WHERE ST_Within(geom, district_polygons.geom)),
        n_res_bldg_green_space = (SELECT COUNT(*) FROM residential_building_polygons WHERE ST_DWithin(geom, (SELECT geom FROM green_area_polygons), 300)),
        a_district = ST_Area(geom),
        area_open_space = (SELECT SUM(ST_Area(geom)) FROM open_area_polygon WHERE ST_Within(geom, district_polygons.geom)),
        area_private_space = (SELECT SUM(ST_Area(geom)) FROM private_public_area_polygons WHERE ST_Within(geom, district_polygons.geom) AND type = 'private'),
        area_public_space = (SELECT SUM(ST_Area(geom)) FROM private_public_area_polygons WHERE ST_Within(geom, district_polygons.geom) AND type = 'public'),
        area_green_space = (SELECT SUM(ST_Area(geom)) FROM green_area_polygons WHERE ST_Within(geom, district_polygons.geom)),
        area_crown_coverage = (SELECT SUM(ST_Area(geom)) FROM tree_crown_polygons WHERE ST_Within(geom, district_polygons.geom)),
        area_crown_coverage_public_space = (SELECT SUM(ST_Area(geom)) FROM tree_crown_polygons WHERE ST_Within(geom, (SELECT geom FROM private_public_area_polygons WHERE type = 'public')))
""")